In [2]:
using JuMP
using ECOS
using GLPK
using MAT
using SparseArrays
using DelimitedFiles
using LinearAlgebra
using MathOptInterface

In [3]:
# read inputs
vars = matread("R2T3.mat")
S = vars["S"]
u1 = SparseArrays.sparsevec(vars["u1"]["nzind"], vars["u1"]["nzval"], vars["u1"]["n"])
l1 = SparseArrays.sparsevec(vars["l1"]["nzind"], vars["l1"]["nzval"], vars["l1"]["n"])

13543-element SparseVector{Float64, Int64} with 924 stored entries:
  [10   ]  =  -0.953761
  [26   ]  =  -0.549852
  [42   ]  =  -0.266553
  [59   ]  =  -0.0106824
  [66   ]  =  -4.0496e-15
  [119  ]  =  -0.290455
  [153  ]  =  -2.48151
  [167  ]  =  -1.46579
  [174  ]  =  -1.65689e-16
  [179  ]  =  -0.413619
           ⋮
  [13216]  =  -0.329301
  [13253]  =  3.54692e-15
  [13259]  =  -1.48781
  [13347]  =  -0.401346
  [13348]  =  -4.52421e-16
  [13397]  =  -0.0570207
  [13434]  =  -1.2646
  [13490]  =  3.06018e-15
  [13493]  =  -4.81149e-16
  [13495]  =  2.40271e-16
  [13539]  =  9.02637e-17

In [4]:
# define model linear
model = JuMP.Model(ECOS.Optimizer)

# define the problem
size_of_v = 13543
@variable(model, v[1:size_of_v])  # define variable v
@variable(model, v_abs[1:size_of_v])
@constraint(model, v_abs .>= v)
@constraint(model, v_abs .>= -v)
@constraint(model, Array(l1) .<= v .<= Array(u1))  # l1 <= v <= u1
@constraint(model, Matrix(S) * v .== 0)  # Sv = 0
@objective(model, Min, sum(v_abs[i] for i in 1:size_of_v));  # min ||v||_1

In [177]:
# solve the l1 approximation LP problem defined above
optimize!(model)
println(termination_status(model))
print(primal_status(model))
starting_v = JuMP.value.(v)

OPTIMAL
FEASIBLE_POINT
ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.388e-17  -8.274e+02  +1e+05  9e-01  6e-01  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  -8.890e+00  -2.557e+02  +8e+04  5e-01  2e-01  2e+00  1e+00  0.6442  3e-01   1  1  1 |  0  0
 2  +2.533e+01  -1.132e+02  +5e+04  3e-01  9e-02  2e+00  9e-01  0.5194  3e-01   1  1  1 |  0  0
 3  +4.646e+01  -4.547e+00  +2e+04  2e-01  3e-02  6e-01  4e-01  0.6420  1e-01   1  1  1 |  0  0
 4  +5.504e+01  +2.556e+01  +1e+04  1e-01  2e-02  3e-01  2e-01  0.4874  2e-01   1  1  1 |  0  0
 5  +5.963e+01  +3.819e+01  +1e+04  8e-02  1e-02  2e-01  2e-01  0.4486  4e-01   1  1  1 |  0  0
 6  +6.303e+01  +4.788e+01  +7e+03  6e-02  8e-03  1e-01  1e-01  0.4494  3e-01   1  1  1 |  0  0
 7  +6.501e+01  +5.427e+01  +5e+03  4e-02  5e-03  7e-02  9e-02  0.4782  4e-01   1  1  1 |  0  0
 8  +6.694e+01  +6.027e+

13543-element Vector{Float64}:
 -1.0474020049080297e-13
  2.0424725928953433e-14
 -4.495482584118648e-16
 -1.57973086623857e-15
  1.0459308404424476e-13
 -2.637485305554223e-14
  3.0937036886472596e-15
 -7.165906621426457e-14
 -6.902018075509321e-14
 -3.853066093182615e-14
 -8.57193744545051e-15
  3.2094767396488956e-14
  9.86026660361096e-14
  ⋮
  1.1985006645735129e-14
 -1.3946890247353041e-16
 -4.7555339442821214e-14
  2.2946469964857577e-13
  2.292586396579232e-13
  1.39692882649357e-16
 -1.504046635784054e-14
 -1.193558158630768e-13
  5.994387775423151e-14
 -7.433235877679912e-14
  2.48600771546879e-13
 -5.5098167041640776e-14

In [45]:
function sgn(x)
    if x < 0
        return -1
    else
        return 1
    end
end

sgn (generic function with 1 method)

In [173]:
# now lets use the iteration method to strengthen the output
delta = 0.0001
n = 1.5
num_iteration = 10
for iteration in 1:num_iteration
    print("doing iteration ")
    println(iteration)
    
    # decrement n
    n -= 0.5 / num_iteration
    
    w = zeros(size_of_v)
    for i in 1:size_of_v
        w[i] = delta ^ n / (delta ^ n + abs(starting_v[i])^n)
        if starting_v[i] < 0
            w[i] = -w[i]
        end
        if l1[i] > 0 || u1[i] < 0
            w[i] = 0
        end
        # w_i = sgn(v_i) * delta^n / (delta^n + v_i^n)
    end
    
    # define model
    model = JuMP.Model(ECOS.Optimizer)

    # define the problem
    @variable(model, v[1:size_of_v])  # define variable v
    @constraint(model, Array(l1) .<= v .<= Array(u1))  # l1 <= v <= u1
    @constraint(model, Matrix(S) * v .== 0)  # Sv = 0
    for i in 1:size_of_v
        @constraint(model, sgn(starting_v[i]) * v[i] >= 0)
        if abs(starting_v[i]) < 0.0000000001
            @constraint(model, v[i] == starting_v[i])
        end
    end
    @objective(model, Min, sum(v[i] * w[i] for i in 1:size_of_v))  # min ||v||_1
    optimize!(model)
    println(termination_status(model))
    println(primal_status(model))
    starting_v = JuMP.value.(v)
end

doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3

ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +4.015e-02  -8.543e+02  +7e+04  9e-01  3e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  +2.174e-02  -1.020e+02  +2e+04  2e-01  3e-02  1e+00  4e-01  0.9223  2e-01   1  1  1 |  0  0
 2  +1.729e-03  -9.967e+00  +2e+03  2e-02  3e-03  1e-01  5e-02  0.8896  1e-02   1  1  1 |  0  0
 3  +3.775e-04  -8.963e-01  +2e+02  2e-03  3e-04  9e-03  5e-03  0.9067  2e-03   1  1  1 |  0  0
 4  +2.792e-04  -1.958e-02  +4e+00  4e-05  6e-06  2e-04  1e-04  0.9776  1e-04   1  1  1 |  0  0
 5  +2.403e-04  -4.238e-03  +1e+00  1e-05  1e-06  3e-05  2e-05  0.8343  7e-02   1  1  1 |  0  0
 6  +1.565e-04  -1.647e-03  +4e-01  4e-06  5e-07  1e-05  1e-05  0.7242  2e-01   1  1  1 |  0  0
 7  +9.515e-05  -3.533e-04  +1e-01  1e-06 

doing iteration 6
OPTIMAL
FEASIBLE_POINT
doing iteration 7
8646  2e-02   1  1  1 |  0  0
 3  +8.066e-04  -1.045e+00  +3e+02  3e-03  3e-04  7e-03  6e-03  0.9533  2e-02   1  1  1 |  0  0
 4  +8.249e-04  -5.977e-02  +2e+01  1e-04  2e-05  4e-04  4e-04  0.9417  9e-04   1  1  1 |  0  0
 5  +5.471e-04  -4.638e-02  +1e+01  1e-04  1e-05  3e-04  3e-04  0.3363  3e-01   1  1  1 |  0  0
 6  +5.482e-04  -2.265e-02  +5e+00  6e-05  7e-06  6e-05  1e-04  0.9163  4e-01   1  1  1 |  0  0
 7  +3.576e-04  -2.736e-03  +7e-01  8e-06  9e-07  8e-06  2e-05  0.8739  8e-03   1  1  1 |  0  0
 8  +3.128e-04  -8.685e-04  +3e-01  3e-06  3e-07  3e-06  6e-06  0.7197  1e-01   1  1  1 |  0  0
 9  +2.987e-04  -4.099e-04  +2e-01  2e-06  2e-07  1e-06  4e-06  0.5917  3e-01   1  1  1 |  0  0
10  +3.079e-04  -1.360e-04  +9e-02  1e-06  1e-07  5e-07  2e-06  0.8486  5e-01   2  1  1 |  0  0
11  +2.883e-04  +6.892e-05  +5e-02  6e-07  7e-08  2e-07  1e-06  0.6778  3e-01   2  1  1 |  0  0
12  +2.773e-04  +1.584e-04  +2e-02  3e-07  4e-0

OPTIMAL
FEASIBLE_POINT
e+01  3e-04  3e-05  7e-04  7e-04  0.3407  3e-01   1  1  1 |  0  0
 6  +1.848e-03  -3.693e-02  +9e+00  1e-04  1e-05  1e-04  2e-04  0.8713  2e-01   1  1  1 |  0  0
 7  +1.569e-03  -7.964e-03  +2e+00  2e-05  3e-06  3e-05  5e-05  0.7634  1e-02   1  1  1 |  0  0
 8  +1.394e-03  -2.056e-03  +8e-01  9e-06  1e-06  8e-06  2e-05  0.8544  2e-01   1  1  1 |  0  0
 9  +1.428e-03  -1.778e-03  +7e-01  9e-06  1e-06  6e-06  2e-05  0.3296  7e-01   1  1  1 |  0  0
10  +1.350e-03  -2.751e-04  +4e-01  4e-06  5e-07  3e-06  9e-06  0.5318  7e-02   1  1  1 |  0  0
11  +1.321e-03  +3.585e-04  +2e-01  3e-06  3e-07  2e-06  5e-06  0.5087  2e-01   1  1  1 |  0  0
12  +1.295e-03  +9.171e-04  +8e-02  1e-06  1e-07  6e-07  2e-06  0.7077  1e-01   2  1  1 |  0  0
13  +1.275e-03  +1.193e-03  +2e-02  2e-07  2e-08  1e-07  4e-07  0.8222  5e-02   2  1  1 |  0  0
14  +1.269e-03  +1.268e-03  +2e-04  3e-09  3e-10  1e-09  6e-09  0.9888  2e-03   3  1  1 |  0  0
15  +1.269e-03  +1.268e-03  +3e-06  3e-11  4e-1

In [178]:
# now we define an Mixed Integer LP (MILP) that is equivalent to the problem. using GLPK we can solve it an get the best possible solution

# define model MILP
model = JuMP.Model(GLPK.Optimizer)

# define the problem
@variable(model, v[1:size_of_v])  # define variable v
for i in 1:size_of_v
    setvalue(v[i], starting_v[i])
end
@variable(model, b[1:size_of_v])
for i in 1:size_of_v
    @constraint(model, l1[i] * b[i] <= v[i])
    @constraint(model, v[i] <= u1[i] * b[i])
    @constraint(model, b[i] in MathOptInterface.ZeroOne())
end
@constraint(model, Array(l1) .<= v .<= Array(u1))  # l1 <= v <= u1
@constraint(model, Matrix(S) * v .== 0)  # Sv = 0
@objective(model, Min, sum(b[i] for i in 1:size_of_v))  # min ||v||_0

optimize!(model)
println(termination_status(model))
print(primal_status(model))
starting_v = JuMP.value.(v)

just to escape from long print of stupid objective declaration!

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer}. This information will be discarded.
└ @ MathOptInterface.Utilities C:\Users\ahmad\.julia\packages\MathOptInterface\YDdD3\src\Utilities\copy.jl:290


OPTIMAL
FEASIBLE_POINT

13543-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.2254208122349342e-16
 ⋮
 2.437019250581812e-15
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [179]:
# save output
DelimitedFiles.writedlm("output.txt", starting_v)